In [1]:
## Emma Abraham (ea7ga) and Andrew Chaphiv (ac2gq)
import nltk 
import discord
from nltk import word_tokenize,sent_tokenize

from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
#read more on the steamer https://towardsdatascience.com/stemming-lemmatization-what-ba782b7c0bd8
import numpy as np 
import tflearn
import tensorflow as tf
import random
import json
import pickle

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [2]:
with open("intents.json", 'r') as file:
    data = json.load(file)

try:
    with open("data.pickle","rb") as f:
        words, labels, training, output = pickle.load(f)
except:
    words = []
    labels = []
    docs_x = []
    docs_y = []
    for intent in data["intents"]:
        for pattern in intent["patterns"]:
            wrds = nltk.word_tokenize(pattern)
            words.extend(wrds)
            docs_x.append(wrds)
            docs_y.append(intent["tag"])
            
        if intent["tag"] not in labels:
            labels.append(intent["tag"])


    words = [stemmer.stem(w.lower()) for w in words if w != "?"]
    words = sorted(list(set(words)))
    labels = sorted(labels)

    training = []
    output = []
    out_empty = [0 for _ in range(len(labels))]

    for x, doc in enumerate(docs_x):
        bag = []

        wrds = [stemmer.stem(w.lower()) for w in doc]

        for w in words:
            if w in wrds:
               bag.append(1)
            else:
              bag.append(0)
    
        output_row = out_empty[:]
        output_row[labels.index(docs_y[x])] = 1
        
        training.append(bag)
        output.append(output_row)

    training = np.array(training)
    output = np.array(output)
    
    with open("data.pickle","wb") as f:
        pickle.dump((words, labels, training, output), f)



net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)
model.fit(training, output, n_epoch=1000, batch_size = 8,show_metric=True)
model.save("model.tflearn")

try:
    model.load("model.tflearn")
except:
    model.fit(training, output, n_epoch=1000, batch_size = 8,show_metric=True)
    model.save("model.tflearn")


Training Step: 3999  | total loss: 0.19291 | time: 0.004s
| Adam | epoch: 1000 | loss: 0.19291 - acc: 0.9862 -- iter: 24/25
Training Step: 4000  | total loss: 0.17376 | time: 0.006s
| Adam | epoch: 1000 | loss: 0.17376 - acc: 0.9875 -- iter: 25/25
--
INFO:tensorflow:C:\Users\andre\Downloads\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.
INFO:tensorflow:Restoring parameters from C:\Users\andre\Downloads\model.tflearn


In [19]:
import pymongo
import pprint
import pandas as pd 

host_name = "localhost"
port = "27017"

atlas_cluster_name = "sandbox"
atlas_default_dbname = "local"
#atlas_user_name = "m001-student"
#atlas_password = "m001-mongodb-basics"
conn_str = {
    "local" : f"mongodb://{host_name}:{port}/",
#    "atlas" : f"mongodb+srv://{atlas_user_name}:{atlas_password}@{atlas_cluster_name}.zibbf.mongodb.net/{atlas_default_dbname}"
}

client = pymongo.MongoClient(conn_str["local"])
db_name = "chatBoxResponses"
db = client[db_name]
m_collection = db["Media"]

def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
    
    return np.array(bag)



def chat():
    print("Start talking with the bot! (type quit to stop)")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break

        result = model.predict([bag_of_words(inp, words)])[0]
        result_index = np.argmax(result)
        tag = labels[result_index]

        if result[result_index] > 0.7:
            for tg in data["intents"]:
                if "Q1" == tag:
                    responses = m_collection.find_one({'RELEASE_YEAR':2022-20,"Type": "Movie"})
                elif "Q2" == tag:
                    result = m_collection.find({'RELEASE_YEAR':2022-10,"Type": "Movie"})
                    result = list(result)
                    a,b,c=np.random.choice(len(result),size = 3,replace = False)
                    responses = [result[a], result[b], result[c]]
                elif "Q3" == tag:
                    result=m_collection.find({'RELEASE_YEAR':2000,"Type": "Movie"}, sort=[("SCORE", -1)])
                    responses = result[0]["MAIN_GENRE"]
                elif "Q4" == tag:
                    result=m_collection.find({'RELEASE_YEAR':2022-5,"Type": "Movie"}, sort=[("SCORE", -1)])
                    responses = result[0]
                elif "Q5" == tag:
                    result=m_collection.find({'RELEASE_YEAR':2010,"Type": "Movie"}, sort=[("DURATION", -1)])
                    responses = result[0]["DURATION"]
                elif "Q6" == tag:
                    responses = m_collection.find_one({'RELEASE_YEAR':2010,"Type": "Show"}, sort=[("SCORE", -1)])
                elif "Q7" == tag:
                    responses = m_collection.find_one({'RELEASE_YEAR':2008,"Type": "Show"}, sort=[("SCORE", -1)])["MAIN_GENRE"]
                elif "Q8" == tag:
                    responses = m_collection.find_one({"Type": "Show"}, sort=[("SCORE", -1)])
                elif "Q9" == tag:
                    responses = m_collection.find_one({"Type": "Show"}, sort=[("DURATION", -1)])
                elif "Q10" == tag:
                    df = pd.DataFrame(list(m_collection.find()))
                    responses = df["RELEASE_YEAR"].value_counts().idxmax()
            if type(responses) == list:
                print(pprint.pprint(responses))
            else:
                print(responses)
        else:
            print("Try asking these questions:")
            print("What movie was popular 20 years ago?")
            print("What were 3 movies popular 10 years ago?")
            print("What was the genre of the top movie in 2000?")
            print("What movie had the top IMDB rating in the past 5 years?")
            print("What is the longest duration of a movie in 2010?")
            print("What was the top show on Netflix 5 years ago?")
            print("What was the genre of the highest-rated show in 2008?")
            print("What show had the top IMBD rating?")
            print("Which show had the longest duration?")
            print("What year had the most top movies?")
chat()






Start talking with the bot! (type quit to stop)


You:  quit
